# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data, cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coxen hole,16.3244,-86.5366,30.60,62,0,4.12,HN,1690413068
1,1,fortuna,40.5982,-124.1573,22.18,67,0,6.17,US,1690413068
2,2,al jawf,29.5000,38.7500,29.82,19,8,3.52,SA,1690413068
3,3,ilulissat,69.2167,-51.1000,13.01,50,0,4.12,GL,1690412878
4,4,coyhaique,-45.5752,-72.0662,-0.03,93,100,1.54,CL,1690413068


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria:
# max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Wind Speed"] < 4.5)
                                       & (city_data_df["Cloudiness"] <= 0 )
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,85,sereflikochisar,38.9393,33.5386,25.00,31,0,2.12,TR,1690412958
166,166,vila velha,-20.3297,-40.2925,21.64,87,0,2.06,BR,1690412987
303,303,cifteler,39.3831,31.0392,25.38,24,0,1.98,TR,1690413095
305,305,sault ste. marie,46.5168,-84.3333,23.31,87,0,3.09,CA,1690412939
456,456,houlton,46.1262,-67.8403,26.23,59,0,2.57,US,1690413116
465,465,kasserine,35.1676,8.8365,24.05,50,0,3.00,TN,1690413116
539,539,karakocan,38.9558,40.0386,24.43,30,0,1.64,TR,1690413122
553,553,baynjiwayn,35.6183,45.9541,23.50,24,0,2.34,IQ,1690413125
582,582,zuwarah,32.9312,12.0820,26.54,74,0,4.23,LY,1690413130


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[[
            "City",
            "Country", 
            "Lat", 
            "Lng", 
            "Humidity"
            ]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
85,sereflikochisar,TR,38.9393,33.5386,31,
166,vila velha,BR,-20.3297,-40.2925,87,
303,cifteler,TR,39.3831,31.0392,24,
305,sault ste. marie,CA,46.5168,-84.3333,87,
456,houlton,US,46.1262,-67.8403,59,
465,kasserine,TN,35.1676,8.8365,50,
539,karakocan,TR,38.9558,40.0386,30,
553,baynjiwayn,IQ,35.6183,45.9541,24,
582,zuwarah,LY,32.9312,12.0820,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apikey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"] 
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sereflikochisar - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
cifteler - nearest hotel: No hotel found
sault ste. marie - nearest hotel: No hotel found
houlton - nearest hotel: No hotel found
kasserine - nearest hotel: No hotel found
karakocan - nearest hotel: No hotel found
baynjiwayn - nearest hotel: No hotel found
zuwarah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
85,sereflikochisar,TR,38.9393,33.5386,31,No hotel found
166,vila velha,BR,-20.3297,-40.2925,87,No hotel found
303,cifteler,TR,39.3831,31.0392,24,No hotel found
305,sault ste. marie,CA,46.5168,-84.3333,87,No hotel found
456,houlton,US,46.1262,-67.8403,59,No hotel found
465,kasserine,TN,35.1676,8.8365,50,No hotel found
539,karakocan,TR,38.9558,40.0386,30,No hotel found
553,baynjiwayn,IQ,35.6183,45.9541,24,No hotel found
582,zuwarah,LY,32.9312,12.0820,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)